<img src="uva_seal.png">  

## Spark Getting Started

### University of Virginia
### DS 5110: Big Data Systems
### Last Updated: January 23, 2023

---  

### SOURCES
Learning Spark, First Edition

Chapter 1: Introduction to Data Analysis with Spark  
Chapter 2: Getting Started

### OBJECTIVES
-  Spark background
-  Setup and installation
-  Basic concepts
-  Minimal code examples
-  Running Spark: Interactive Session
-  Running Spark: Command Line

### CONCEPTS

- Cluster: a set of connected computers (nodes)

- Functional programming

- SparkSession - single point of entry to interact w Spark functionality

- Resilient Distributed Datasets (RDDs) - Spark’s fundamental abstraction for distributed data and computation

- Dataset

- Driver Program - contains application main function, defines RDDs on cluster, applies operations to them.

- Worker Node or Executor - the units that perform tasks

---

### Spark Benefits

- Designed to be fast  
no waiting around for hours, need to work interactively with data  

- Designed to handle big data

- General Purpose  
Unlike Hadoop, several modules in one place: 
  - Machine learning
  - SQL queries
  - Streaming
  - Graph analytics


- Caching is possible, so intermediate data can be stored in memory on workers

- Highly accessible: simple APIs to Python, Java, Scala, R, SQL  
Integrates w other Big Data tools such as Hadoop, Cassandra  
Can access HDFS data, Amazon S3, and others

---

### Set up a Spark Session with Minimal Parameters

- use local machine as master
- name the app

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local") \
        .appName("pyspark_test") \
        .getOrCreate()

In [4]:
# print info about the session
spark

In [5]:
# need this for working with RDDs
sc = spark.sparkContext

---

### RDDs (Resilient Distributed Datasets)

RDD objects are the most basic abstraction in Spark. They have these properties:

- **resilient**: list of dependencies instructs Spark how the RDD is constructed from inputs.  
  In the event the RDD is compromised, Spark can recreate it from dependencies.
  
- uses `partitions` for storing pieces of the data. Spark automatically partitions RDDs and distributes the partitions across nodes in the cluster.
  
- **distributed**: placing the partitions across nodes in the cluster allows for storing massive datasets that wouldn't fit on a single machine. 

**RDD History**

Before Spark 2.0, the main programming interface of Spark was the *Resilient Distributed Dataset (RDD)*.  

Starting with Spark 2.0, the *Dataset* and *DataFrame* objects were released. They are built up from RDDs.  
We work with DataFrames later.

The RDD interface is still supported  

---

### Computing

We will look at several examples.

**Example 1: Read lines from text file**

In [6]:
data_filename = 'README.txt'

In [7]:
lines = sc.textFile(data_filename)

In [8]:
lines.count()

41

In [9]:
lines.first()

'Apache Spark'

In [10]:
lines.collect()

['Apache Spark',
 'Spark is a unified analytics engine for large-scale data processing. It provides high-level APIs in Scala, Java, Python, and R, and an optimized engine that supports general computation graphs for data analysis. It also supports a rich set of higher-level tools including Spark SQL for SQL and DataFrames, MLlib for machine learning, GraphX for graph processing, and Structured Streaming for stream processing.',
 '',
 'https://spark.apache.org/',
 '',
 'Jenkins Build AppVeyor Build PySpark Coverage',
 '',
 'Online Documentation',
 'You can find the latest Spark documentation, including a programming guide, on the project web page. This README file only contains basic setup instructions.',
 '',
 'Building Spark',
 'Spark is built using Apache Maven. To build Spark and its example programs, run:',
 '',
 './build/mvn -DskipTests clean package',
 '(You do not need to do this if you downloaded a pre-built package.)',
 '',
 'You can build Spark using more than one thread by u

In [11]:
type(lines.collect())

list

In [12]:
lines.collect()[1]

'Spark is a unified analytics engine for large-scale data processing. It provides high-level APIs in Scala, Java, Python, and R, and an optimized engine that supports general computation graphs for data analysis. It also supports a rich set of higher-level tools including Spark SQL for SQL and DataFrames, MLlib for machine learning, GraphX for graph processing, and Structured Streaming for stream processing.'

In [13]:
type(lines.collect()[0])

str

**Example 2: Text Search  - apply filter and print all lines containing “Spark”**

In [17]:
spark_lines = lines.filter(lambda x: "Spark" in x)

In [18]:
# return list of first 5 records
spark_lines.take(5)   

['Apache Spark',
 'Spark is a unified analytics engine for large-scale data processing. It provides high-level APIs in Scala, Java, Python, and R, and an optimized engine that supports general computation graphs for data analysis. It also supports a rich set of higher-level tools including Spark SQL for SQL and DataFrames, MLlib for machine learning, GraphX for graph processing, and Structured Streaming for stream processing.',
 'Jenkins Build AppVeyor Build PySpark Coverage',
 'You can find the latest Spark documentation, including a programming guide, on the project web page. This README file only contains basic setup instructions.',
 'Building Spark']

In [19]:
type(spark_lines)

pyspark.rdd.PipelinedRDD

**Example 3: Word Count**

In [26]:
# Read the file into an RDD
lines = sc.textFile(data_filename)

In [27]:
type(lines)

pyspark.rdd.RDD

In [28]:
words = lines.flatMap(lambda x: x.split())

In [29]:
words.take(5)

['Apache', 'Spark', 'Spark', 'is', 'a']

In [30]:
wordcounts = words.map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y) \
                  .map(lambda x:(x[1],x[0])) \
                  .sortByKey(False)

In [31]:
wordcounts.take(10)

[(13, 'the'),
 (11, 'Spark'),
 (7, 'to'),
 (6, 'a'),
 (6, 'for'),
 (6, 'and'),
 (6, 'can'),
 (6, 'run'),
 (5, 'is'),
 (5, 'using')]

**TRY FOR YOURSELF (UNGRADED EXERCISES)**

1) Convert the Word Count example into a function called `word_count()`.

The function assumes you have already read in the text file into `lines`.  
It should take two inputs: 
- `lines`  this is the RDD containing text
- `num_records`  this is the number of wordcount pairs to return

It should output a list of the `num_records` most frequent word count pairs.  
Enter the code for `word_count()` in the cell below.

In [ ]:
## definition of word_count()


Now test that `word_count()` returns the expected result.  
Also insure that the output type is a list.

In [ ]:
## test function: word_count()
## calling type(output) should return a list


**SOLUTIONS**

In [ ]:
def word_count(lines, num_records):
    
    wordcounts = lines.flatMap(lambda x: x.split()) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(lambda x,y:x+y) \
                  .map(lambda x:(x[1],x[0])) \
                  .sortByKey(False)
    
    return(wordcounts.take(num_records))

In [ ]:
out = word_count(lines, 10)
out

In [ ]:
type(out)